In [1]:
import pandas as pd
import re
import numpy as np
import os
import win32com.client
import win32com
import os
import sys
from datetime import datetime

In [2]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
sent_items = outlook.Folders('rpa@amazon.com').Folders("Sent Items")

In [3]:
info=pd.DataFrame()

In [4]:
# convert email data to df, use n for sample
n = 1
for item in sent_items.items:
    #n+=1
    temp = []
    b = item.body
    b=b.replace('\n','')
    b=b.split('\r')
    #if n == 100:
    #   break
    if len (b) >= 8:
        index=[0,2,3,4,5,6,7,8]
        for x in index:
            if x == 0:
                temp.append(b[x].replace('Hi,',''))
            else:
                temp.append(b[x].replace(b[x].split(':')[0]+':',''))
        temp=pd.DataFrame(temp).T
        info=pd.concat([info,temp])
    else:
        pass

In [5]:
info.columns=['Login','Robot_ID','Email_Received_Time','Start_Time','End_Time','Run_Time','Task_Info','Status']

In [6]:
info['Task_ID'] = info['Status']
info['Source'] = info['Task_Info']
info['Target'] = info['Task_Info']

In [7]:
info = info[info['Login'] != 'Hello,']
info = info[info['Login'] != '']
info = info[info['Login'] != 'RPA']

In [8]:
# get task id from task info and convert run time
for index, row in info.iterrows(): 
    if 'Succeeded' in row['Task_ID']:
        row['Task_ID'] = row['Task_ID'].strip().split(' ')[1].strip()
    else:
        row['Task_ID'] = np.nan
    if ':' in row['Run_Time']:
        row['Run_Time'] = float (row['Run_Time'].split(':')[0])+float (row['Run_Time'].split(':')[1])/60+float(row['Run_Time'].split(':')[2])/3600

In [9]:
# get source & target data from task info
for index, row in info.iterrows(): 
    if row['Task_ID'] == '3':
        try:
            row['Source'] = row['Task_Info'].split('_')[1].strip().upper()
            row['Target'] = row['Task_Info'].split('_')[2].strip().upper()
        except:
            print (row['Task_Info'])
    elif row['Task_ID'] == '4':
        row['Source'] = "US"
        row['Target'] = "AU"
    elif  row['Task_ID'] == '2' or row['Task_ID'] == '6' or row['Task_ID'] == '11':
        row['Source'] = row['Task_Info'].split('_')[1].strip()[:2].upper()
        row['Target'] = row['Task_Info'].split('_')[1].strip()[3:].upper()
    elif row['Task_ID'] == '5':
        row['Source'] = np.nan
        row['Target'] = np.nan
    elif row['Task_ID'] == '8':
        row['Source'] = row['Task_Info'].strip()[:2].upper()
        row['Target'] = row['Task_Info'].strip()[3:5].upper()
    elif row['Task_ID'] == '9':
        row['Source'] = np.nan
        row['Target'] = np.nan
    elif row['Task_ID'] == '10':
        row['Source'] = row['Task_Info'].split('_')[1].upper()
        row['Target'] = row['Task_Info'].split('_')[2].upper()
    else: 
        row['Source'] = np.nan
        row['Target'] = np.nan

In [ ]:
#clean rows with empty 'Start_Time'
info = info[info['Start_Time'] != '']

In [10]:
info.drop_duplicates().to_csv("test.csv", index = False)